In [3]:
#### LOZANO GARCIA EDUARDO ALEJNADRO  elozango1600@alumno.ipn.mx
#### UPIBI IPN- Mexcio City, MEXICO , BMSIS Visting Scholar Washington D.C., United States 
#### 11/20/2020


import cv2
import serial 
import time 
import numpy as np
import matplotlib.pyplot as plt
import os

# Remote Wheel Chair Control with Ouccular Movement:

The following project is the implementation of a wheel chair controller through the camara of a computer. The camara captures the occular movements and sends the data through pyserial to Arduino. This data is sent, through bluetooth, to another arduino, which controlls a protoype to scale of a wheel chair.

## Circuits:

### Car Circuit (with slave Bluetooth)

<img src="images\Car_circuit_bb.png" alt="Drawing" style="width: 500px;"/>

### Control Circuit (with Master Bluetooth)

<img src="images\Car_control_eye_and_voice_bb.png" alt="Drawing" style="width: 500px;"/>


## Bluetooth Configuration:

INSTRUCTIONS:

UPLOAD Bluetooth_configuration.ino to SLAVE and MASTER. BEFORE conecting arduino push the button of the HC05, connect arduino and let go the button after 5 seconds (It should blink slowly) 

**Bluetooth_configuration.ino:**
```c
#include <SoftwareSerial.h>
SoftwareSerial mySerial(10, 11); // RX, TX

void setup() {
  Serial.begin(9600);      // Open serial communications and wait for port to open:
  mySerial.begin(38400);   // set the data rate for the SoftwareSerial port
  
  while (!Serial) {         
   ;                       // Wait for serial communiactaion 
  }
  
  // mySerial.println("Hello, world?");
}

void loop() { // run over and over
  if (mySerial.available()) {
    Serial.write(mySerial.read());
 
  }
  if (Serial.available()) {
    mySerial.write(Serial.read());
  }
 
}
```
**SLAVE Configuration:**

``` c
AT+UART=38400,0,0     // Set BAUD RATE
AT+ROLE=0             // Set Role to Slave (0=Slave, 1=Master)
AT+NAME= "NAME"       // Set the name you wish to have
AT+PSWD=1234          // Set Pin

AT+ADDR?              // Retrive the MAC Adress of the Slave 
(EXAMPLE ADDRESS)      
+ADDR:21:13:5d5f2     // SLAVE ADDRES   (0021,13,05d5f2) Remember to add 00's
AT+RESET              // Finish Slave setup
 
MASTER Configuration
AT+UART=38400,0,0      // Set BAUD RATE
AT+ROLE=1              // Set Role to Master (0=Slave, 1=Master)
AT+CMODE=0             // Set Conectivity by STATIC ADDRES
AT+BIND=0021,13,05d5f2 // PUT MAC Slave's MAC Address with "," instead of ":" and add 0's
AT+RESET               // Finish Master setup
```
Notice that if we connect by a static addresss we will not require the name and the password of the device in order to establish conection

CODE OVERVIEW (ADD ? to make query add =value to set parameter)
```c
AT+ADDR     // RETRURNS default address
AT+VERSION  // RETURNS version 
AT+NAME     // RETURNS NAME
AT+PSWD     // RETRUNS Device Password
AT+UART     // RETRUNS BAUD RATE
AT+ROLE     // RETURNS ROLE
AT+RESET    // EXISTS CONFIGURATION
AT+ORGL     // ESTABLISH FACTORY  VALUES (BACK TO NEW) 
```

## Arduino Code

Once The circuit is built and the HC05 modules are configured as masters and slaves we are ready to do the next step: Upload the Arduino code. For this it is important to take in mind that master (conected to the arduino Mega) will be communictating through pyserail with this jupyternotebook. The slave HC05 is connected to the arduino UNO, which is incharge of ontrooling the wheel chair


### Master Arduino Code (Interacts with this jupyternotebook)

```c 
// Code For Master Bluetooth Module 
#include <SoftwareSerial.h>       // PACKAGE FOR SERIAL COMMUNICATION
char dataFrompython;              // Variable for Data recived from python
int Transmit;                     // Transmit value  
SoftwareSerial Bluetooth(10, 11); // RX (CONNECTED TO DEVICE TX), TX (CONNECTED TO DEVICE RX)

void setup() {
  Serial.begin(9600);    // ARDUINO BAUD RATE
  Bluetooth.begin(38400); // BLUETOOTH BAUD RATE
  
}

void loop() {
  if (Serial.available() > 0){
    dataFrompython =  Serial.read();
    //-----------------STOP --------------------//
    if (dataFrompython == '0') {
      Transmit=0; 
     }
    //-----------------  Left  --------------------//
    if (dataFrompython == '1') {
      Transmit=1;
      }
    //----------------- Right --------------------//
    if (dataFrompython == '2') {
     Transmit=2;
      }
      
    //----------------- Front --------------------//
    if (dataFrompython == '3') {
     Transmit=3;
      }

    //----------------- Back --------------------//
    if (dataFrompython == '4') {
      Transmit=4;
      }
  }
  // SEND DATA VIA BLUETOOTH
  
  Bluetooth.write(Transmit); // Send Value from master
 
 
  //Serial.println(Transmit); // 
  delay(5);
  
}

```

### Slave Arduino Code (Recives the data via bluetooth and controls wheel chair)

```c
int IN2 = 2;             // RIGHT FRONT
int ENA = 5;             // RIGHT ENABLE

int IN3 = 8;             // LEFT BACK
int IN4 = 7;             // LEFT FRONT  
int ENB = 6;             // LEFT ENABLE
int dataFromMaster = 0;  // VARIABLE TO STORE DATA FROM RECIVER (Associated with control of axis)
int RawData;             // Varaibale to store incomeing data from bluetooth
int SPEED = 100;         // PWM
// INFO FROM MASTER
// 0 HALT
// 1 LEFT
// 2 RIGHT
// 3 FRONT
// 4 BACK 
#include <SoftwareSerial.h>
SoftwareSerial  Bluetooth(10, 11); // RX, TX
void setup() 
{
  pinMode(IN1,OUTPUT); 
  pinMode(IN2,OUTPUT);
  pinMode(IN3,OUTPUT);
  pinMode(IN4,OUTPUT);

  pinMode(ENA,OUTPUT);
  pinMode(ENB,OUTPUT);
  Serial.begin(9600);    // ARDUINO BAUD RATE
  Bluetooth.begin(38400); // BLUETOOTH BAUD RATE
}

void loop() {
  // ---------------- RECIVE DATA FROM MASTER ---------------
  if (Bluetooth.available()){
    RawData = Bluetooth.read();   // Reads the data from the serial port and store it in dataFromMaster variable
    if (  RawData < 5 ) {
        dataFromMaster=RawData;
    }
    else if ( RawData > 69){
      SPEED=RawData;
      
    }

     // ---------------- HAULT --------------- 
     if (dataFromMaster ==  0) 
     {
       Serial.println("Halut");
       digitalWrite(ENA,LOW);
       digitalWrite(IN1,LOW);
       digitalWrite(IN2,LOW);
       digitalWrite(ENB,LOW);
       digitalWrite(IN3,LOW);
       digitalWrite(IN4,LOW);
       delay(5);

           
     }
     
      // 1 LEFT
      else if (dataFromMaster ==  1) 
      {
       Serial.println("LEFT");
       digitalWrite(ENA,LOW);
       digitalWrite(IN1,LOW);
       digitalWrite(IN2,LOW);
       analogWrite(ENB,SPEED);
       digitalWrite(IN3,LOW);
       digitalWrite(IN4,HIGH);
       delay(5);

     
    
     
      
      }
      
      // 2 RIGHT
      else if (dataFromMaster ==  2) 
      {
      // Serial.println("RIGHT");

      analogWrite(ENA,SPEED);
      digitalWrite(IN1,LOW);
      digitalWrite(IN2,HIGH);
      digitalWrite(ENB,LOW);
      digitalWrite(IN3,LOW);
      digitalWrite(IN4,LOW);
      delay(5);

      }
      
      // 3 FRONT
      else if (dataFromMaster ==  3)
      {
      // Serial.println("FRONT");
       analogWrite(ENA,SPEED);
       digitalWrite(IN1,LOW);
       digitalWrite(IN2,HIGH);
       analogWrite(ENB,SPEED);
       digitalWrite(IN3,LOW);
       digitalWrite(IN4,HIGH);
       delay(5);

      }

      // 4 BACK
     else if (dataFromMaster ==  4) 
     {
    //Serial.println("BACK");
     analogWrite(ENA,SPEED);
     digitalWrite(IN1,HIGH);
     digitalWrite(IN2,LOW);
     analogWrite(ENB,SPEED);
     digitalWrite(IN3,HIGH);
     digitalWrite(IN4,LOW);
     delay(5);

     }
     
     
    delay(5);
   
  }
} 
```

## Create function

In [4]:
def Arduino_Car_Controll_w_eye(COM,b,t,serial_status):
    if serial_status:
        ser =  serial.Serial(COM,baudrate=b,timeout=t) # Initiate Communication
        time.sleep(2)                                  # Wait 2 Seconds so that communication load correctly

    ## This is just an example to ilustrate how to use Haar Cascades in order to detect objects (LIVE) ##
    face = cv2.CascadeClassifier('Haarcascade/haarcascade_frontalface_default.xml') # Face Haar Cascade loading
    eye = cv2.CascadeClassifier('Haarcascade/haarcascade_eye.xml')                  # Eye  Haar Cascade Loading
    path = "Data_set_eye"                                                           # Path to Store Photos
    frame_id = 0                                                                    # Frame Id
    vid = cv2.VideoCapture(0)                                                       # Define a video capture object 
    status = True                                                                   # Initalize status
    width = 100
    height = 100
    dimensions=(width,height)
    font = cv2.FONT_HERSHEY_SIMPLEX
    while(status): 
        status, frame = vid.read()                             # Capture the video frame by frame 
        frame2 = np.copy(frame)                                # Copy frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)         # Convert to gray scale
        face_info = face.detectMultiScale(gray, 1.3, 10)        # Get face infromation
        if face_info  != ():                                   # If face was capture
            (x,y,w,h) = face_info[0]                           # unpack information
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),1) # Draw rectangle
            cropped_face = gray[y:y+h, x:x+w]                  # Crop face (color)   
            eye_info = eye.detectMultiScale(cropped_face,1.3,20)      # eye info

            if  eye_info != (): 
                for (xe,ye,we,he)  in eye_info:
                    xn =x+xe
                    yn =y+ye
                    cv2.rectangle(frame,(xn,yn),(xn+we,yn+he),(255,0,255),1) 

            if eye_info == ():
                 command = "Stop"
                 if serial_status:
                     ser.write(b"0")                                # Send 1 to arduino
                

            elif  eye_info.__len__() == 2:
                command = "Forward"
                if serial_status:
                    ser.write(b"3")                                # Send 1 to arduino

            elif eye_info.__len__() == 1:
                resta  = (x - xe)/x
                if resta < 0.7:
                    command = "Left"
                    if serial_status:
                        ser.write(b"1")    

                elif resta > 0.8:
                    command = "Right"
                    if serial_status:
                        ser.write(b"2")   

        elif face_info  == ():  
             command = "Stop"
             if  serial_status:
                 ser.write(b"0")   

        cv2.putText(frame,command,(20,20), font, 1,(255,255,0),2) 
        cv2.imshow('frame',  frame)                            # Display the resulting frame 
        wait_key = cv2.waitKey(1) & 0xFF                       # StQQore Waitkey object
        if wait_key  == ord('q'):                              # If q is pressed
            break                                              # Break while loop



    vid.release()              # After the loop release the cap object 
    cv2.destroyAllWindows()    # Destroy all the windows 
    if serial_status:
        ser.close()                # Close connection with Arduino 

## Use Function

In [5]:
COM = "COM5"                                       # Select COM
b=9600                                             # Select BAUDRATE
t=1                                                # Selecet Timeout
serial_status = False                              # Activate arduino 
Arduino_Car_Controll_w_eye(COM,b,t,serial_status)  # Call function

c:\users\eduardo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
c:\users\eduardo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
c:\users\eduardo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [ ]:
Q